1. **Category**: A table for different categories of instruments (e.g., String Instruments, Wind Instruments).
2. **Instrument**: A table for instruments that belong to a category.
3. **Manufacturer**: A table for the manufacturers of instruments.
3. **InstrumentManufacturer**: A join table to handle the many-to-many relationship between instruments and manufacturers.

## Load Libraries

In [18]:
from eralchemy2 import render_er
from sqlalchemy.orm import relationship, sessionmaker, declarative_base
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table

In [19]:
# Create an SQLite database
engine = create_engine('sqlite:///synthetic.db', echo=False) # Set to True to see SQL commands

# Define a base class for our models
Base = declarative_base()

# Many-to-Many Relationship Table
instrument_manufacturer = Table('instrument_manufacturer', Base.metadata,
    Column('instrument_id', Integer, ForeignKey('instruments.id'), primary_key=True),
    Column('manufacturer_id', Integer, ForeignKey('manufacturers.id'), primary_key=True)
)

In [20]:
# Define the Category table
class Category(Base):
    __tablename__ = 'categories'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

    instruments = relationship("Instrument", back_populates="category")

# Define the Instrument table
class Instrument(Base):
    __tablename__ = 'instruments'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    category_id = Column(Integer, ForeignKey('categories.id'))
    
    category = relationship("Category", back_populates="instruments")
    manufacturers = relationship("Manufacturer", secondary=instrument_manufacturer, back_populates="instruments")

# Define the Manufacturer table
class Manufacturer(Base):
    __tablename__ = 'manufacturers'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    
    instruments = relationship("Instrument", secondary=instrument_manufacturer, back_populates="manufacturers")

# Create all tables
Base.metadata.create_all(engine)

## Step 2: Creating a Session and Adding Data

Now, let's add some data to these tables.

In [21]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Create categories
string_category = Category(name="String Instruments")
wind_category = Category(name="Wind Instruments")

# Create instruments
violin = Instrument(name="Violin", category=string_category)
flute = Instrument(name="Flute", category=wind_category)

# Create manufacturers
yamaha = Manufacturer(name="Yamaha")
stradivarius = Manufacturer(name="Stradivarius")

# Establish relationships between instruments and manufacturers
violin.manufacturers.append(stradivarius)
violin.manufacturers.append(yamaha)
flute.manufacturers.append(yamaha)

# Add to session
session.add(string_category)
session.add(wind_category)
session.add(violin)
session.add(flute)
session.add(yamaha)
session.add(stradivarius)

# Commit the session to write to the database
session.commit()

In [22]:
import pandas as pd

# Load each table into a Pandas DataFrame
categories_df = pd.read_sql_table('categories', engine)
instruments_df = pd.read_sql_table('instruments', engine)
manufacturers_df = pd.read_sql_table('manufacturers', engine)
instrument_manufacturer_df = pd.read_sql_table('instrument_manufacturer', engine)

# Display the contents of each DataFrame
print("Categories Table:")
display(categories_df)

print("\nInstruments Table:")
display(instruments_df)

print("\nManufacturers Table:")
display(manufacturers_df)

print("\nInstrument_Manufacturer Table:")
display(instrument_manufacturer_df)

Categories Table:


,id,name
0,1,String Instruments
1,2,Wind Instruments



Instruments Table:


,id,name,category_id
0,1,Violin,1
1,2,Flute,2



Manufacturers Table:


,id,name
0,1,Stradivarius
1,2,Yamaha



Instrument_Manufacturer Table:


,instrument_id,manufacturer_id
0,1,1
1,1,2
2,2,2


## Generate ER

In [23]:
# Generate ER diagram and save it as a file
render_er('sqlite:///synthetic.db', 'synthetic_db_erd.png')